# **Importamos las librerias de interes**

In [2]:
import numpy as np

import pandas as pd

from google.colab import drive



# **Extraemos la data**

In [3]:

# conectamos al drive
drive.mount('/content/drive')

# Definimos el directorio
dir_data_kaggle='./drive/MyDrive/alicorp/data'


# extraemos la data

df_user=pd.read_csv(filepath_or_buffer=dir_data_kaggle+'/train.csv',sep='|')

df_product=pd.read_csv(filepath_or_buffer=dir_data_kaggle+'/RentabilidadProduct.csv',sep='|')

Mounted at /content/drive


In [4]:
#  le echamos un ojo a user

df_user.sample(10)

,fecha_compra,customer_id,product_id,type_id,business_id,channel_id,cantidad_venta,monto_venta_transf,peso_venta_transf
2003307,2023-01-09,2382,987,BO,21,A,1.0,31.401149,2.640163
2009408,2023-01-10,5461,976,GA,54,V,1.0,13.898489,2.605424
252957,2022-03-17,10070,165,GA,51,B,60.0,1.722475,0.138956
374438,2022-04-08,9420,563,BO,21,V,3.0,2.052495,0.130271
1363304,2022-09-24,638,452,BO,21,V,1.0,6.484611,0.636881
1117865,2022-08-15,8269,209,PA,31,B,2.0,44.897245,4.631865
2073066,2023-01-20,1803,182,BO,21,V,3.0,4.796875,0.303966
301948,2022-03-25,8180,11,BO,21,V,6.0,11.863364,1.736949
663711,2022-05-28,4221,193,PA,31,B,2.0,72.332359,5.789831
1317139,2022-09-17,10834,450,BO,21,V,2.0,2.142237,0.156325


In [5]:
# le ehcamos un ojo a product

df_product.sample(10)

,product_id,family_id,negocio_id,category_id,tier_id,utilidad_bruta_transf
773,221,1155,20,168,1,-0.311225
923,76,1239,20,100,3,-2.598370
79,922,1097,20,187,1,0.622395
279,247,1154,20,168,2,-0.205324
655,855,1275,20,131,2,-0.304009
782,348,1389,20,112,4,-0.312252
259,453,1051,20,115,3,-0.189012
645,778,1141,20,166,3,-0.303834
680,497,1249,20,173,2,-0.304430
794,147,1027,21,181,1,-0.314800


In [6]:
# revisamos si hay datos Nans en user

df_user.isna().any()

fecha_compra          False
customer_id           False
product_id            False
type_id               False
business_id           False
channel_id            False
cantidad_venta        False
monto_venta_transf    False
peso_venta_transf     False
dtype: bool

In [7]:
# revisamos si hay datos Nans en product

df_product.isna().any()

product_id               False
family_id                False
negocio_id               False
category_id              False
tier_id                  False
utilidad_bruta_transf    False
dtype: bool

# **Arreglo de df_product**

In [8]:
# veamos que hay productos en df_user que no estan en df_product

missing_product_id=[]

unique_id_product_user=pd.unique(df_user['product_id'])

unique_id_product=pd.unique(df_product['product_id'])


for product_id in unique_id_product_user:
  if product_id in unique_id_product:
    pass
  else:
    missing_product_id.append(product_id)


print(f'Hay {len(missing_product_id)} ids faltantes')

Hay 111 ids faltantes


In [9]:
# creamos el dataframe para anexar a df_product

df_missing_product_id=pd.Series(missing_product_id)

columns_product=df_product.columns

In [10]:
columns_product

Index(['product_id', 'family_id', 'negocio_id', 'category_id', 'tier_id',
       'utilidad_bruta_transf'],
      dtype='object')

In [11]:
# etiquetas auxiliares

label_aux=columns_product.drop(['product_id','utilidad_bruta_transf'])

# entramos al loop

for label in label_aux:

  stuffed_serie=pd.Series([df_product[label].min()-1]*len(missing_product_id))

  df_missing_product_id=pd.concat(objs=[df_missing_product_id,stuffed_serie],
                                  axis=1)

# la columna final la hacemos a mano

utilidad_serie=pd.Series([0]*len(missing_product_id))

df_missing_product_id=pd.concat(objs=[df_missing_product_id,utilidad_serie]
                                ,axis=1)

# arreglamos las columnas


df_missing_product_id.columns=columns_product

In [12]:
# vemos el dataframe de relleno

df_missing_product_id

,product_id,family_id,negocio_id,category_id,tier_id,utilidad_bruta_transf
0,977,1002,19,99,0,0
1,1021,1002,19,99,0,0
2,973,1002,19,99,0,0
3,1009,1002,19,99,0,0
4,1008,1002,19,99,0,0
...,...,...,...,...,...,...
106,1041,1002,19,99,0,0
107,975,1002,19,99,0,0
108,976,1002,19,99,0,0
109,971,1002,19,99,0,0


In [13]:
# concatenmaos df_product y df_missing_product_id

df_product=pd.concat(objs=[df_product,df_missing_product_id],axis=0,ignore_index=True)

In [14]:
# vemos el resultado

df_product

,product_id,family_id,negocio_id,category_id,tier_id,utilidad_bruta_transf
0,881,1291,20,131,4,10.378017
1,805,1224,20,131,3,10.177925
2,892,1089,20,146,1,10.008243
3,841,1224,20,131,3,7.698362
4,830,1089,20,131,1,7.353678
...,...,...,...,...,...,...
1043,1041,1002,19,99,0,0.000000
1044,975,1002,19,99,0,0.000000
1045,976,1002,19,99,0,0.000000
1046,971,1002,19,99,0,0.000000


In [15]:
# comprobamos que todo product_id que esta en df_user esta en df_product

missing_product_id=[]

unique_id_product_user=pd.unique(df_user['product_id'])

unique_id_product=pd.unique(df_product['product_id'])


for product_id in unique_id_product_user:
  if product_id in unique_id_product:
    pass
  else:
    missing_product_id.append(product_id)


print(f'Hay {len(missing_product_id)} ids faltantes')

Hay 0 ids faltantes


# **Arreglo de df_user**

In [16]:
df_time=df_user['fecha_compra'].astype('datetime64')

In [18]:
df_transf_time=pd.concat(objs=[df_time,
                               pd.Series(df_time.dt.day_of_week),
                               pd.Series(df_time.dt.day)//8,
                               pd.Series(df_time.dt.month)],
                         axis=1)

df_transf_time.columns=['date','day_of_week','week_of_month','month']

In [19]:
# vemos la data transformada

df_transf_time

,date,day_of_week,week_of_month,month
0,2022-01-31,0,3,1
1,2022-01-31,0,3,1
2,2022-01-31,0,3,1
3,2022-01-31,0,3,1
4,2022-01-31,0,3,1
...,...,...,...,...
2167843,2023-02-04,5,0,2
2167844,2023-02-04,5,0,2
2167845,2023-02-04,5,0,2
2167846,2023-02-04,5,0,2


In [20]:
# ahora hacemos drop y concatenamos

df_user.drop(columns=['fecha_compra'],inplace=True)

df_user=pd.concat(objs=[df_user,df_transf_time],axis=1,ignore_index=False)


In [21]:
# vemos el resultado final

df_user

,customer_id,product_id,type_id,business_id,channel_id,cantidad_venta,monto_venta_transf,peso_venta_transf,date,day_of_week,week_of_month,month
0,7157,510,BO,21,V,1.0,0.153431,0.005211,2022-01-31,0,3,1
1,7157,512,BO,21,V,1.0,0.153431,0.005211,2022-01-31,0,3,1
2,9382,510,BO,21,V,1.0,0.153431,0.005211,2022-01-31,0,3,1
3,9382,512,BO,21,V,1.0,0.153431,0.005211,2022-01-31,0,3,1
4,9382,525,BO,21,V,1.0,0.153431,0.005790,2022-01-31,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2167843,646,667,PA,31,B,55.0,2085.986380,796.101767,2023-02-04,5,0,2
2167844,13758,667,PA,31,B,60.0,2339.780728,868.474655,2023-02-04,5,0,2
2167845,3111,673,GA,31,B,64.0,2460.348170,926.372966,2023-02-04,5,0,2
2167846,10520,679,PA,31,B,100.0,3531.912728,1447.457759,2023-02-04,5,0,2


# **Exportamos la data**

In [22]:
# nos llevamos el df user

df_user.to_csv(path_or_buf=dir_data_kaggle+'/df_user.csv', index=False)

# nos llevamos el df

df_product.to_csv(path_or_buf=dir_data_kaggle+'/df_product.csv', index=False)